In [1]:
import pymysql
import pandas as pd
from datetime import datetime

In [53]:
import tqdm
import os
import cv2
import numpy as np

def concat_origin_img(white_img, black_img):
    white_img = 255 - white_img

    img_sum = np.zeros((white_img.shape[0],white_img.shape[1],3), dtype=np.uint8)
    img_sum[:,:,0] = white_img
    img_sum[:,:,1] = black_img
    
    return img_sum

def find_circles(img_origin):
    try:
        img = img_origin[:,:,1]
        #가우시안필터
        gaus = cv2.GaussianBlur(img, (5, 5), 7)

        #적응형이진화
        thres = cv2.adaptiveThreshold(gaus, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 5)
        thres = cv2.GaussianBlur(thres, (5, 5), 7)
        
        circles = cv2.HoughCircles(thres, cv2.HOUGH_GRADIENT, 1, 1200, param1 = 250, param2 = 60, minRadius = 720, maxRadius = 800)
    except Exception as e:
        print(e)
        return False, img    
    try:
        if len(circles) > 0:
            for i in circles[0]:
                cx = i[0]
                cy = i[1]
                radius = i[2]
    except:
        print('fail to find circle!')
        cx = img_origin.shape[1]//2
        cy = img_origin.shape[0]//2
        radius = 740

    #이미지 ROI 추출
    margin = 40 # 여백
    s_radius = radius
    x_min = max(0, int(cx - s_radius - margin))  # Ensure x_min is not less than 0
    y_min = max(0, int(cy - s_radius - margin))  # Ensure y_min is not less than 0
    x_max = min(img_origin.shape[1], int(cx + s_radius + margin))  # Ensure x_max does not exceed image width
    y_max = min(img_origin.shape[0], int(cy + s_radius + margin))  # Ensure y_max does not exceed image height

    img_roi = img_origin[y_min:y_max,x_min:x_max]
    if img_roi.size == 0: # If the ROI is empty, provide some error message or handle this case
        return False, img
    else:
        img_roi = cv2.resize(img_roi,(1500,1500))

    return True, img_roi

def copy_from_nas(port_number, set_date, file_list):
    # 나스에서 조회용
    nas_base = r'\\contactlensEB/Home/contactlensEB_backup/'
    if port_number == 1:
        nas_path = nas_base + '/images_copy/original_images'
    elif port_number == 2:
        nas_path = nas_base + '/images_copy2/original_images'
    elif port_number == 3:
        nas_path = nas_base + '/images_copy3/original_images'

    #몇호기인지
    set_machine = nas_path
    
    print(set_date)
    
    #전처리완료된 이미지 저장용 폴더
    save_folder = r'E:\claassifier\images_sec_renew\additional_target2'
    file_list = [x[0] for x in file_list]
    file_list = ["_".join(x.split("_")[1:]) for x in file_list]

    for file in tqdm.tqdm(file_list):
        file_path = set_machine + '/'+ set_date + '/' + file
        white_path = file_path.replace('1.jpg','0.jpg')
        b_img = cv2.imread(file_path,0)
        w_img = cv2.imread(white_path,0)
        
        concated_img = concat_origin_img(white_img=w_img, black_img=b_img)
        is_success, pic = find_circles(concated_img)

        cv2.imwrite(save_folder+ '/' + file, pic)

In [ ]:
## 작업분류 엑셀 E:\project\렌즈\작업일지
csv_path = 'E:\project\렌즈\작업일지\work.csv'
df = pd.read_csv(csv_path, encoding='cp949')

q_df = df[['모델명', '호수', '날짜', '시작시간', '완료시간']]
q_df

conn = pymysql.connect(host='175.212.10.234', user='cgacAdmin', password='gac81-344', db='contactlensEB', charset='utf8',port=3307)
conn2 = pymysql.connect(host='175.212.10.234', user='cgacAdmin', password='gac81-344', db='contactlensEB', charset='utf8',port=3308)
cursor = conn.cursor()
cursor2 = conn2.cursor()

for i,row in enumerate (q_df.itertuples()):
    date = row.날짜
    date_obj = datetime.strptime(date, '%m월%d일')
    month, day = date_obj.month, date_obj.day
    start_time = row.시작시간
    time_obj = datetime.strptime(start_time, '%H:%M')
    hour, minute = time_obj.hour, time_obj.minute
    start_date = datetime(year=2023, month=month, day=day, hour=hour, minute=minute)

    end_time =row.완료시간
    time_obj = datetime.strptime(end_time,'%H:%M')
    hour, minute = time_obj.hour, time_obj.minute
    end_date = datetime(year=2023, month=month,day=day,hour=hour, minute=minute)
    #시간내총수량 = base_query

    base_query = f'''
                SELECT COUNT(*) FROM result
                WHERE workDate BETWEEN '{str(start_date)}' and '{str(end_date)}'
                '''
    if row.호수 == 2:
        cursor.execute(base_query)
        b_res = cursor.fetchall()
    else:
        cursor2.execute(base_query)
        b_res = cursor2.fetchall()
    
    total_count = int(b_res[0][0])
    #2차분류_불량
    query = base_query + f'''
            AND (defectstep='1D' or defectstep = '2D')
            '''
    if row.호수 == 2:
        cursor.execute(query)
        res = cursor.fetchall()
    else:
        cursor2.execute(query)
        res = cursor2.fetchall()
    
    defect_sec_count = int(res[0][0])

    sec_f_list_query = f'''
                    SELECT filename FROM result
                    WHERE workDate BETWEEN '{str(start_date)}' and '{str(end_date)}'
                    AND (defectstep='1D' or defectstep = '2D')
                    '''
    if row.호수 == 2:
        cursor.execute(sec_f_list_query)
        fres = cursor.fetchall()
    else:
        cursor2.execute(sec_f_list_query)
        fres = cursor2.fetchall()

    copy_from_nas(port_number=row.호수, 
                  set_date=str(start_date).replace("-","")[:8],
                  file_list=list(fres))

    q_df.loc[[i],['DB총수량']]=total_count
    q_df.loc[[i],['2차분류_불량']]=defect_sec_count
    q_df.loc[[i],['2차분류불량율']]=round(defect_sec_count/total_count, 2)

conn.close()
conn2.close()

# q_df.to_csv('temp.csv')



In [51]:
q_df.head()

,모델명,호수,날짜,시작시간,완료시간,DB총수량,2차분류_불량,2차분류불량율
0,tint,2,7월27일,9:51,17:39,9895.0,0.0,0.00
1,DA6247,2,8월11일,13:29,14:13,1666.0,214.0,0.13
2,DA4276,2,8월17일,9:28,9:52,1180.0,67.0,0.06
3,DA4076,2,8월11일,10:38,11:47,1697.0,406.0,0.24
4,DA4076,2,8월17일,9:53,10:20,1299.0,121.0,0.09


In [52]:
q_df

,모델명,호수,날짜,시작시간,완료시간,DB총수량,2차분류_불량,2차분류불량율
0,tint,2,7월27일,9:51,17:39,9895.0,0.0,0.00
1,DA6247,2,8월11일,13:29,14:13,1666.0,214.0,0.13
2,DA4276,2,8월17일,9:28,9:52,1180.0,67.0,0.06
3,DA4076,2,8월11일,10:38,11:47,1697.0,406.0,0.24
4,DA4076,2,8월17일,9:53,10:20,1299.0,121.0,0.09
...,...,...,...,...,...,...,...,...
94,60TT,3,8월25일,11:50,13:50,2200.0,23.0,0.01
95,55TT,3,8월25일,13:50,14:24,1208.0,27.0,0.02
96,55TT,3,8월25일,14:25,16:45,2575.0,38.0,0.01
97,DA3352,3,8월17일,13:37,14:25,280.0,4.0,0.01


In [86]:
import shutil
import warnings
warnings.filterwarnings('ignore')

add1_path = r'E:\claassifier\images_sec_renew\additional_target'
add2_path = r'E:\claassifier\images_sec_renew\additional_target2'
save_path = r'E:\claassifier\images_sec_renew\final'

add1_csv = pd.read_csv(add1_path + '/result.csv')
add2_csv = pd.read_csv(add2_path + '/result.csv')

add1_csv = add1_csv[add1_csv['good'] == 1]
add2_csv = add2_csv[add2_csv['defect'] == 1]
final_csv = pd.DataFrame(columns=add1_csv.columns)
final_csv = final_csv[['id', 'good', 'defect', 'no_lens']]

print('양품데이터 : ', len(add1_csv))
print('불량데이터 : ', len(add2_csv))
print(final_csv)

#양품 과 불량이 같이 체크되어있는것 확인
print('중복체크된 행 : ', len(add1_csv.loc[(add1_csv['good'] == 1) & (add1_csv['defect'] == 1)]))
print('중복체크된 행 : ', len(add1_csv.loc[(add1_csv['good'] == 1) & (add1_csv['no_lens'] == 1)]))
print('중복체크된 행 : ', len(add2_csv.loc[(add2_csv['good'] == 1) & (add2_csv['defect'] == 1)]))
print('중복체크된 행 : ', len(add2_csv.loc[(add2_csv['good'] == 1) & (add2_csv['no_lens'] == 1)]))

good_bar = tqdm.tqdm(add1_csv.itertuples(), total=len(add1_csv))
#good은 좌우플립만 수행
for rows in good_bar:
    if os.path.isfile(add1_path +'/'+ rows.id) and rows.good == 1:
        #오리지널 그대로 복사
        shutil.copy(add1_path + '/' + rows.id, save_path +'/'+ rows.id)
        new_row = pd.Series([rows.id, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)

        #양품데이터는 플립1회 수행(x2)
        img = cv2.imread(add1_path +'/'+ rows.id)
        img_flip_lr = cv2.flip(img, 1)
        lr_filename = 'lr'+rows.id
        cv2.imwrite(save_path + '/' + lr_filename , img_flip_lr)
        new_row = pd.Series([lr_filename, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)
        
defect_bar = tqdm.tqdm(add2_csv.itertuples(), total=len(add2_csv))
for rows in defect_bar:
    if os.path.isfile(add2_path +'/'+ rows.id) and rows.defect == 1:
        #오리지널 그대로 복사
        shutil.copy(add2_path + '/' + rows.id, save_path +'/'+ rows.id)
        new_row = pd.Series([rows.id, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)

        #불량데이터는 플립2회 수행(x4)
        img = cv2.imread(add2_path +'/'+ rows.id)
        img_flip_lr = cv2.flip(img, 1)
        lr_filename = 'lr'+rows.id
        cv2.imwrite(save_path + '/' + lr_filename , img_flip_lr)
        new_row = pd.Series([lr_filename, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)

        img_flip_ud = cv2.flip(img, 0)
        ud_filename = 'ud'+rows.id
        cv2.imwrite(save_path + '/' + ud_filename , img_flip_ud)
        new_row = pd.Series([ud_filename, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)

        img_flip_lrud = cv2.flip(img_flip_ud, 1)
        lrud_filename = 'lrud'+rows.id
        cv2.imwrite(save_path + '/' + lrud_filename , img_flip_lrud)
        new_row = pd.Series([lrud_filename, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        final_csv = final_csv.append(new_row, ignore_index=True)

print(final_csv)

양품데이터 :  3800
불량데이터 :  1324
Empty DataFrame
Columns: [id, good, defect, no_lens]
Index: []
중복체크된 행 :  0
중복체크된 행 :  0
중복체크된 행 :  0
중복체크된 행 :  0


100%|██████████| 1324/1324 [03:03<00:00,  7.23it/s]

                                 id good defect no_lens
0          20230718182225_0_R_1.jpg    1      0       0
1        lr20230718182225_0_R_1.jpg    1      0       0
2          20230718182225_1_R_1.jpg    1      0       0
3        lr20230718182225_1_R_1.jpg    1      0       0
4          20230718182225_3_R_1.jpg    1      0       0
...                             ...  ...    ...     ...
12891  lrud20230825164802_4_R_1.jpg    0      1       0
12892      20230825165854_4_R_1.jpg    0      1       0
12893    lr20230825165854_4_R_1.jpg    0      1       0
12894    ud20230825165854_4_R_1.jpg    0      1       0
12895  lrud20230825165854_4_R_1.jpg    0      1       0

[12896 rows x 4 columns]


In [87]:
final_csv.to_csv("dataset_sec_add.csv", index=False)

In [101]:
final_csv['sum'] = final_csv[['good', 'defect', 'no_lens']].sum(axis=1)

array([1.])

In [107]:
add_dataset = pd.read_csv('dataset_sec_add.csv')
print(len(add_dataset))
original_dataset = pd.read_csv('dataset_sec.csv')
print(len(original_dataset))
con_df = pd.concat([add_dataset, original_dataset], ignore_index=True)

print(len(con_df))
print(con_df.columns)

con_df.to_csv('dataset_sec_full.csv', index=False)

12896
119091
131987
Index(['id', 'good', 'defect', 'no_lens'], dtype='object')


In [3]:
full_dataset = pd.read_csv('dataset_sec_full.csv')
print("총이미지개수 : ",len(full_dataset))
full_dataset.sum()

총이미지개수 :  131987


id         20230718182225_0_R_1.jpglr20230718182225_0_R_1...
good                                                   67600
defect                                                 61682
no_lens                                                 2705
dtype: object

In [26]:
import fnmatch
import shutil
import warnings
warnings.filterwarnings('ignore')

add_dataset = pd.read_csv('dataset_sec_add.csv')
print(len(add_dataset))
asdfasf_df = pd.read_excel(r'E:\claassifier'+'/실금.xlsx')
addsave_path = r'E:\claassifier\images_sec_renew\20231019_add'

for rows in asdfasf_df.itertuples():
    d_name = rows._1[:8]
    d_path = r"\\113.198.138.232\Home\contactlensEB_backup\images_copy3\modified_images" + '/' + d_name
    
    for f in os.listdir(d_path):
        count = 0
        if fnmatch.fnmatch(f, '*'+rows._1+ '_1.jpg'):
            f_name = f
            if count != 0:
                print('something wrong')
                break
            count += 1
        
    # '/'+ rows._1 + '_1.jpg'
    shutil.copy(d_path+'/'+f_name, addsave_path+'/'+f_name)
    new_row = pd.Series([f_name,0, 1, 0], index=add_dataset.columns)
    add_dataset = add_dataset.append(new_row, ignore_index=True)
    
print(len(add_dataset))
        # new_row = pd.Series([lr_filename, rows.good, rows.defect, rows.no_lens], index=final_csv.columns)
        # final_csv = final_csv.append(new_row, ignore_index=True)

12896
13139


In [28]:
add_dataset
add_dataset.to_csv("dataset_sec_add2.csv", index=False)

In [29]:
add_dataset = pd.read_csv('dataset_sec_add2.csv')
print(len(add_dataset))
original_dataset = pd.read_csv('dataset_sec.csv')
print(len(original_dataset))
con_df = pd.concat([add_dataset, original_dataset], ignore_index=True)

print(len(con_df))
print(con_df.columns)

con_df.to_csv('dataset_sec_full2.csv', index=False)

13139
119091
132230
Index(['id', 'good', 'defect', 'no_lens'], dtype='object')
